<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/Stock_NeurIPS2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Task Description](#0)
* [2. Install Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. A List of Python Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download and Preprocess Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5. Build Market Environment in OpenAI Gym-style](#4)  
    * [5.1. Data Split](#4.1)  
    * [5.3. Environment for Training](#4.2)    
* [6. Train DRL Agents](#5)
* [7. Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
  

<a id='0'></a>
# Part 1. Task Discription

We train a DRL agent for stock trading. This task is modeled as a Markov Decision Process (MDP), and the objective function is maximizing (expected) cumulative return.

We specify the state-action-reward as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes many features and learns by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 consituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.


The data for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Install Python Packages

<a id='1.1'></a>
## 2.1. Install packages


In [2]:
## install finrl library
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [3]:
import datetime as dt
import logging
trial_ = dt.datetime.now().strftime('%Y_%M_%d_%H_%M')
logging.basicConfig(filename=f"results/ddpg/example_{trial_}.log",  level=logging.DEBUG)


<a id='1.2'></a>
## 2.2. A list of Python packages 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

import sys
sys.path.append("../finrl")

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from env_stocktrading_pair_trading import StockPairTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import itertools

c:\Users\Administrador\AppData\Local\Programs\Python\Python38\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [5]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



<a id='2'></a>
# Part 3. Download Data
Yahoo Finance provides stock data, financial news, financial reports, etc. Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** in FinRL-Meta to fetch data via Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).



-----
class YahooDownloader:
    Retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()


In [6]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-07-31'

In [7]:
TRAIN_START_DATE = '2010-10-19'
TRAIN_END_DATE = '2020-09-11' # 70% of All data set size (2556 aprox.)
TRADE_START_DATE = '2020-09-11'
TRADE_END_DATE = '2022-10-18'
PAIR_TRADE = ['AON','MMC']
# PAIR_TRADE = ['ACN','NDAQ']

In [8]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = PAIR_TRADE).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (6040, 8)


In [9]:
# print(config_tickers.DOW_30_TICKER)

In [10]:
df.shape

(6040, 8)

In [11]:
df.sort_values(['date','tic'],ignore_index=True).head()
# Structure : date (%Y-%m-%d), open, high, low, close, volume, tic, day

,date,open,high,low,close,volume,tic,day
0,2010-10-19,39.430000,39.910000,39.180000,34.570560,2891000,AON,1
1,2010-10-19,24.090000,24.540001,24.059999,19.009434,4187500,MMC,1
2,2010-10-20,39.590000,39.709999,39.419998,34.710636,2148600,AON,2
3,2010-10-20,24.440001,24.799999,24.340000,19.297812,2942600,MMC,2
4,2010-10-21,39.779999,40.360001,39.590000,35.069553,2539100,AON,3


# Part 4: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

In [12]:
INDICATORS = ['close_5_sma']
# INDICATORS = []
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=False,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


In [13]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [14]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,close_5_sma
0,2010-10-19,AON,39.430000,39.910000,39.180000,34.570560,2891000.0,1.0,34.570560
1,2010-10-19,MMC,24.090000,24.540001,24.059999,19.009434,4187500.0,1.0,19.009434
2,2010-10-20,AON,39.590000,39.709999,39.419998,34.710636,2148600.0,2.0,34.640598
3,2010-10-20,MMC,24.440001,24.799999,24.340000,19.297812,2942600.0,2.0,19.153623
4,2010-10-21,AON,39.779999,40.360001,39.590000,35.069553,2539100.0,3.0,34.783583
5,2010-10-21,MMC,24.830000,25.190001,24.690001,19.601780,3957400.0,3.0,19.303008
6,2010-10-22,AON,40.139999,40.189999,39.959999,35.034550,1525200.0,4.0,34.846325
7,2010-10-22,MMC,25.139999,25.290001,25.030001,19.632952,3355300.0,4.0,19.385494
8,2010-10-25,AON,40.290001,40.400002,40.169998,35.279655,1691200.0,0.0,34.932991
9,2010-10-25,MMC,25.290001,25.370001,25.030001,19.578390,2373300.0,0.0,19.424073


<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [15]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

4982
1058


In [16]:
train.tail()

,date,tic,open,high,low,close,volume,day,close_5_sma
2488,2020-09-08,MMC,114.540001,114.739998,112.139999,109.770035,2689500.0,1.0,112.180569
2489,2020-09-09,AON,200.139999,205.600006,199.350006,200.025742,1155200.0,2.0,199.986432
2489,2020-09-09,MMC,114.449997,117.809998,114.349998,112.933144,1843600.0,2.0,112.426268
2490,2020-09-10,AON,204.360001,206.399994,201.759995,198.473663,1594500.0,3.0,198.477615
2490,2020-09-10,MMC,116.760002,117.500000,115.580002,111.830406,1859200.0,3.0,111.787843


In [17]:
trade.head()

,date,tic,open,high,low,close,volume,day,close_5_sma
0,2020-09-11,AON,202.259995,203.070007,199.500000,197.972702,1189200.0,4.0,198.023788
0,2020-09-11,MMC,115.959999,116.449997,114.860001,111.994850,1023800.0,4.0,111.633075
1,2020-09-14,AON,203.330002,204.419998,201.679993,198.385300,1053600.0,0.0,197.870551
1,2020-09-14,MMC,116.839996,118.040001,116.470001,113.948822,1199300.0,0.0,112.095451
2,2020-09-15,AON,202.279999,203.919998,201.679993,198.424561,1013400.0,1.0,198.656393


In [18]:
INDICATORS

['close_5_sma']

In [19]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 2, State Space: 7


In [20]:
print('>> ',train.shape)
train.head()

>>  (4982, 9)


,date,tic,open,high,low,close,volume,day,close_5_sma
0,2010-10-19,AON,39.430000,39.910000,39.180000,34.570560,2891000.0,1.0,34.570560
0,2010-10-19,MMC,24.090000,24.540001,24.059999,19.009434,4187500.0,1.0,19.009434
1,2010-10-20,AON,39.590000,39.709999,39.419998,34.710636,2148600.0,2.0,34.640598
1,2010-10-20,MMC,24.440001,24.799999,24.340000,19.297812,2942600.0,2.0,19.153623
2,2010-10-21,AON,39.779999,40.360001,39.590000,35.069553,2539100.0,3.0,34.783583


In [21]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

action_space = 3 # [-1:sell, 0:hold, 1:buy] # ==== OP1 
env_kwargs = {
    "hmax": 0,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": action_space, # ==== OP1 
    # "action_space":stock_dimension, # ==== OP2
    "reward_scaling": 1e-4,
    "print_verbosity" : 1,
    "model_name" : "test_pair_trading",
    "pair_name":'_'.join(PAIR_TRADE),
}

e_train_gym = StockPairTradingEnv(df = train, **env_kwargs)


In [22]:
train

,date,tic,open,high,low,close,volume,day,close_5_sma
0,2010-10-19,AON,39.430000,39.910000,39.180000,34.570560,2891000.0,1.0,34.570560
0,2010-10-19,MMC,24.090000,24.540001,24.059999,19.009434,4187500.0,1.0,19.009434
1,2010-10-20,AON,39.590000,39.709999,39.419998,34.710636,2148600.0,2.0,34.640598
1,2010-10-20,MMC,24.440001,24.799999,24.340000,19.297812,2942600.0,2.0,19.153623
2,2010-10-21,AON,39.779999,40.360001,39.590000,35.069553,2539100.0,3.0,34.783583
...,...,...,...,...,...,...,...,...,...
2488,2020-09-08,MMC,114.540001,114.739998,112.139999,109.770035,2689500.0,1.0,112.180569
2489,2020-09-09,AON,200.139999,205.600006,199.350006,200.025742,1155200.0,2.0,199.986432
2489,2020-09-09,MMC,114.449997,117.809998,114.349998,112.933144,1843600.0,2.0,112.426268
2490,2020-09-10,AON,204.360001,206.399994,201.759995,198.473663,1594500.0,3.0,198.477615


## Environment for Training



In [23]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [24]:
agent = DRLAgent(env = env_train)

if_using_a2c = True
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = False


### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [25]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c


In [55]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=5000) if if_using_a2c else None

self.state[0] :: 	 1000000
--- >> 	 self.state 1 :  [999980.576133049, 47.722015380859375, 28.374244689941406, 1.0, -1.0, 48.016456604003906, 28.40555305480957]
actions[0] ::  1.0 	B_:1.0	 	S_:-1.0	 Assets: [1.0, -1.0]	 self.reward >>  -0.0004011042320285924 

self.state[0] :: 	 999980.576133049
--- >> 	 self.state 2 :  [999980.576133049, 34.710636138916016, 19.29781150817871, 1.0, -1.0, 34.64059829711914, 19.1536226272583]
actions[0] ::  -0.0 	B_:0	 	S_:0	 Assets: [1.0, -1.0]	 self.reward >>  5.494880676269531e-06 

self.state[0] :: 	 999980.576133049
--- >> 	 self.state 3 :  [999965.0536882781, 35.06955337524414, 19.60177993774414, 2.0, -2.0, 34.783583323160805, 19.303008397420246]
actions[0] ::  1.0 	B_:1.0	 	S_:-1.0	 Assets: [2.0, -2.0]	 self.reward >>  -1.870222550351173e-05 

self.state[0] :: 	 999965.0536882781
--- >> 	 self.state 4 :  [999949.5974228, 35.034549713134766, 19.632951736450195, 3.0, -3.0, 34.8463249206543, 19.385494232177734]
actions[0] ::  1.0 	B_:1.0	 	S_:-1.0	 A

c:\Git\FinRL_cq\tutorials\1-Introduction\env_stocktrading_pair_trading.py:293: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "index" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig(


self.state[0] :: 	 1000117.6296841792
--- >> 	 self.state 10 :  [1000132.9172901094, 34.943931579589844, 19.60177993774414, -4.0, 4.0, 35.34491729736328, 19.578395462036134]
actions[0] ::  -1.0 	B_:1.0	 	S_:-1.0	 Assets: [-4.0, 4.0]	 self.reward >>  -1.9575817498844116e-05 

self.state[0] :: 	 1000132.9172901094
--- >> 	 self.state 11 :  [1000148.240023567, 35.04937744140625, 19.67192268371582, -5.0, 5.0, 35.28485107421875, 19.628276824951172]
actions[0] ::  -1.0 	B_:1.0	 	S_:-1.0	 Assets: [-5.0, 5.0]	 self.reward >>  2.712159709772095e-05 

self.state[0] :: 	 1000148.240023567
--- >> 	 self.state 12 :  [1000163.4974165228, 35.09330749511719, 19.78104019165039, -6.0, 6.0, 35.14778213500976, 19.640747833251954]
actions[0] ::  -1.0 	B_:1.0	 	S_:-1.0	 Assets: [-6.0, 6.0]	 self.reward >>  -0.00031246833076002076 

self.state[0] :: 	 1000163.4974165228
--- >> 	 self.state 13 :  [1000163.4974165228, 36.10375213623047, 20.279850006103516, -6.0, 6.0, 35.22334442138672, 19.760774230957033]
acti

In [56]:
env_train.observation_space

Box([-inf -inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf inf], (7,), float32)

In [57]:
trained_a2c

### Agent 2: DDPG

In [58]:
# DDPG_PARAMS = {"batch_size": 10, "buffer_size": 300, "learning_rate": 0.001, "learning_starts" : 2700}

# agent = DRLAgent(env = env_train)
# model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

# if if_using_ddpg:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/ddpg'
#   new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_ddpg.set_logger(new_logger_ddpg)

In [59]:
# tmp_path

In [60]:
# trained_ddpg = agent.train_model(model=model_ddpg,
#                                  tb_log_name='ddpg',
#                                  total_timesteps=3000) if if_using_ddpg else None


### Agent 3: PPO

In [61]:
# agent = DRLAgent(env = env_train)
# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.01,
#     "learning_rate": 0.00025,
#     "batch_size": 128,
# }
# model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

# if if_using_ppo:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/a2c'
#   new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_ppo.set_logger(new_logger_ppo)

In [62]:
# trained_ppo = agent.train_model(model=model_ppo, 
#                              tb_log_name='ppo',
#                              total_timesteps=50000) if if_using_ppo else None

### Agent 4: TD3

In [63]:
# agent = DRLAgent(env = env_train)
# TD3_PARAMS = {"batch_size": 100, 
#               "buffer_size": 1000000, 
#               "learning_rate": 0.001}

# model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

# if if_using_td3:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/td3'
#   new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_td3.set_logger(new_logger_td3)

In [64]:
# trained_td3 = agent.train_model(model=model_td3, 
#                              tb_log_name='td3',
#                              total_timesteps=30000) if if_using_td3 else None

### Agent 5: SAC

In [65]:
# agent = DRLAgent(env = env_train)
# SAC_PARAMS = {
#     "batch_size": 128,
#     "buffer_size": 5, # 100000,
#     "learning_rate": 0.001,
#     "learning_starts": 100,
#     "ent_coef": "auto_0.1",
# }

# model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

# if if_using_sac:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/sac'
#   new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_sac.set_logger(new_logger_sac)

In [66]:
# trained_sac = agent.train_model(model=model_sac, 
#                              tb_log_name='sac',
#                              total_timesteps=10) if if_using_sac else None

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [67]:
# data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
# insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [68]:
# insample_risk_indicator.vix.describe()

In [69]:
# insample_risk_indicator.vix.quantile(0.996)

In [70]:
# insample_risk_indicator.turbulence.describe()

In [71]:
# insample_risk_indicator.turbulence.quantile(0.996)

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [72]:
# e_trade_gym = StockPairTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
e_trade_gym = StockPairTradingEnv(df = trade, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [73]:
trained_a2c

In [74]:
trained_moedl = trained_a2c
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

self.state[0] :: 	 1000000
--- >> 	 self.state 1 :  [1000085.6678843155, 197.9727020263672, 111.9948501586914, -1.0, 1.0, 198.02378845214844, 111.63307495117188]
actions[0] ::  -1.0 	B_:1.0	 	S_:-1.0	 Assets: [-1.0, 1.0]	 self.reward >>  0.00012314066543476658 

self.state[0] :: 	 1000085.6678843155
--- >> 	 self.state 2 :  [1000169.792027855, 198.3852996826172, 113.94882202148438, -2.0, 2.0, 197.87055053710938, 112.09545135498047]
actions[0] ::  -1.0 	B_:1.0	 	S_:-1.0	 Assets: [-2.0, 2.0]	 self.reward >>  0.00011955088959075511 

self.state[0] :: 	 1000169.792027855
--- >> 	 self.state 3 :  [1000253.1614174423, 198.424560546875, 114.74200439453125, -3.0, 3.0, 198.6563934326172, 113.0898452758789]
actions[0] ::  -1.0 	B_:1.0	 	S_:-1.0	 Assets: [-3.0, 3.0]	 self.reward >>  -0.00012655896607320757 

self.state[0] :: 	 1000253.1614174423
--- >> 	 self.state 4 :  [1000336.84755764, 198.94517517089844, 114.94514465332031, -4.0, 4.0, 198.44028015136718, 113.49224548339843]
actions[0] ::  -1.

In [75]:
df_account_value['date'] = pd.to_datetime(df_account_value['date'], format='%Y-%m-%d')
df_account_value.shape

(529, 2)

In [76]:
plt.figure(figsize=(20,12))
plt.plot(df_account_value['date'],df_account_value['account_value'])
plt.savefig(f"{RESULTS_DIR}/ddpg/performance_Portfolio_trial_{trial_}.png")

In [77]:
df_account_value.tail()

,date,account_value
524,2022-10-11,996288.140515
525,2022-10-12,997330.070095
526,2022-10-13,995428.047537
527,2022-10-14,998894.885570
528,2022-10-17,997465.150885


In [78]:
df_actions['L/S'].value_counts()

-1.0    528
Name: L/S, dtype: int64

In [50]:
# import torch as th

In [51]:
# trained_ddpg.actor.mu[0].weight.shape

In [52]:
# # amount, price_A, price_B, Qty_A, Qty_B, IndTec_A, IndTec_B
# trained_ddpg.actor.mu(
#     th.from_numpy(
#         np.array([1000085.6678843155, 197.9727020263672, 111.9948501586914, -1.0, 1.0, 198.02379760742187, 111.63307647705078])
#     ).float()
# )

In [53]:
# trained_ddpg.actor.mu()

In [54]:
raise Exception('asdfasdfasdf')

Exception: asdfasdfasdf

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])